1. Create a single dataframe with the concatenation of all input csv files, adding a column called country
2. Extract all videos that have no tag.
3. For each channel, determine the total number of views
4. Save all rows with disabled comments and disabled ratings, or that have video_error_or_removed in a new dataframe called excluded, and remove those rows from the original dataframe.
5. Add a like_ratio column storing the ratio between the number of likes and of dislikes
6. Cluster the publish time into 10-minute intervals (e.g. from 02:20 to 02:30)
7. For each interval, determine the number of videos, average number of likes and of dislikes.
8. For each tag, determine the number of videos
Notice that tags contains a string with several tags.

9. Find the tags with the largest number of videos
10. For each (tag, country) pair, compute average ratio likes/dislikes
11. For each (trending_date, country) pair, the video with the largest number of views
12. Divide trending_date into three columns: year, month, day
13. For each (month, country) pair, the video with the largest number of views
14. Read all json files with the video categories
15. For each country, determine how many videos have a category that is not assignable.

0. Import all the packages that need to be used.

In [44]:
import pandas as pd
import numpy as np
import json
import os

                                                  POINT 1
Create a single dataframe with the concatenation of all input csv files, adding a column called country

In [53]:
folder_path = "YoutubeTrends_Data/csv_data/"

files = {
    "CA": "CAvideos.csv.zst",
    "DE": "DEvideos.csv.zst",
    "FR": "FRvideos.csv.zst",
    "GB": "GBvideos.csv.zst",
    "IN": "INvideos.csv.zst",
    "JP": "JPvideos.csv.zst",
    "KR": "KRvideos.csv.zst",
    "MX": "MXvideos.csv.zst",
    "RU": "RUvideos.csv.zst",
    "US": "USvideos.csv.zst"
}

all_dataframes = []

for country_code, filename in files.items():
    full_path = folder_path + filename

    try:
        df = pd.read_csv(full_path, compression='zstd', encoding="utf-8", encoding_errors="ignore")
        df["country"] = country_code
        all_dataframes.append(df)
        print(f"Uploaded: {country_code}")

    except FileNotFoundError:
        print(f"WARNING: Can't find: {full_path}")

if all_dataframes:
    YT_Trends = pd.concat(all_dataframes, ignore_index=True)

    print(f"YT_Trends has a total of {len(YT_Trends)} rows and {len(YT_Trends.columns)} columns")
    print(f"Is the index unique? {YT_Trends.index.is_unique}")

Uploaded: CA
Uploaded: DE
Uploaded: FR
Uploaded: GB
Uploaded: IN
Uploaded: JP
Uploaded: KR
Uploaded: MX
Uploaded: RU
Uploaded: US

Final dataframe:
YT_Trends has a total of 375942 rows and 17 columns
Is the index unique? True


                                                    POINT 2
Extract all videos that have no tag.

In [54]:
no_tag=YT_Trends.loc[YT_Trends["tags"]=="[none]"]
#you can also remove the ".loc" and it works the same

print(f"{len(no_tag)} videos do not have any tag")
print(no_tag)

37698 videos do not have any tag
           video_id trending_date  \
41      JwboxqDylgg      17.14.11   
58      9B-q8h31Bpk      17.14.11   
78      1UE5Dq1rvUA      17.14.11   
86      pmJQ4KwliX4      17.14.11   
98      lHcXhBojpeQ      17.14.11   
...             ...           ...   
375815  VGykknw9eCM      18.14.06   
375819  fAIX12F6958      18.14.06   
375865  gS1DbvHHVH0      18.14.06   
375873  E4c7EE8_IX0      18.14.06   
375938  1h7KV2sjUWY      18.14.06   

                                                    title       channel_title  \
41      Canada Soccer's Women's National Team v USA In...       Canada Soccer   
58      John Oliver Tackles Louis C.K. And Donald Trum...            TV Shows   
78                Taylor Swift Perform Ready For It - SNL          Ken Reactz   
86      LATEST Q POSTS: ROTHSCHILDS, HOUSE OF SAUD, lL...        James Munder   
98                       三屆TVB視帝，拋棄10年青梅竹馬髮妻，為娶小三還不惜與母絕交！               明星百曉生   
...                                 

                                                    POINT 3
For each channel, determine the total number of views

In [55]:
pd.DataFrame(YT_Trends.groupby("channel_title").sum().loc[:,"views"].rename("total_views"))

#count counts how many times the channel appears therefore is not the solution
#I converted it into a dataframe and also renamed the column as "total_views" instead of the original "views"
#could also be ordered with sort_values(ascending=False) ((descending order or either ascending with True))

,total_views
channel_title,
! 세상에 무슨일이,3942977
!!8時だよ面白ネタ大集合,50207
!BTS・TWICE まとめ,7310
!Los amorosos Virales¡,6069
!t Live,240038
...,...
５分でできる DIY,272607
ＢＡＳＨｔｖ,116292
Ｋのフィッシングちゃんねる,37165


                                                            POINT 4
Save all rows with disabled comments and disabled ratings, or that have video_error_or_removed in a new dataframe called excluded, and remove those rows from the original dataframe.

In [56]:
condition = ((YT_Trends["comments_disabled"] == True) & (YT_Trends["ratings_disabled"] == True)) | (YT_Trends["video_error_or_removed"] == True)

excluded = YT_Trends[condition].copy()

YT_Trends = YT_Trends[~condition].copy()

print(f"{len(excluded)} rows were removed from YT_Trends and moved to 'excluded'. YT_Trends has now these many rows: {len(YT_Trends)}, while originally it counted 375942 rows")

2620 rows were removed from YT_Trends and moved to 'excluded'. YT_Trends has now these many rows: 373322, while originally it counted 375942 rows


                                                        POINT 5
Add a like_ratio column storing the ratio between the number of likes and of dislikes

In [57]:
YT_Trends['like_ratio'] = (YT_Trends.loc[:,"likes"]/(YT_Trends.loc[:,"dislikes"].replace(0, np.nan)))
#in this case we handle the case in which the number of dislikes is 0. which mathematically leads to an infinite result

YT_Trends.loc[:, ["title", "like_ratio"]].sort_values(by="like_ratio", ascending=False)
#in this case we have to add the by unlike point 3 because in this case we have a DataFrame, while in that one we were dealing with a Series

,title,like_ratio
331431,Alone in the Game | AT&T AUDIENCE Network,11688.000000
296366,[Dance Practice] 몬스타엑스 (MONSTA X) - DRAMARAMA,3550.750000
221150,[Dance Practice] 몬스타엑스 (MONSTA X) - DRAMARAMA,3550.500000
259304,[BT21] Meet BT21,3233.857143
113540,Swing - Rivage (Prod. Le Motel),3177.000000
...,...,...
358439,Camera Goes on Japanese Sushi Conveyor Belt Sh...,NaN
360544,ULTRA LIVE presents Ultra Music Festival 2018 ...,NaN
363543,Coachella 2018 LIVE Channel 1,NaN
363744,Coachella 2018 LIVE Channel 1,NaN


                                                        POINT 6
Cluster the publish time into 10-minute intervals (e.g. from 02:20 to 02:30)

In [7]:
YT_Trends["publish_time"] = pd.to_datetime(YT_Trends["publish_time"])
YT_Trends["time_clustering"] = YT_Trends["publish_time"].dt.floor("10min")

print(YT_Trends["time_clustering"])

0        2017-11-10 17:00:00+00:00
1        2017-11-13 17:00:00+00:00
2        2017-11-12 19:00:00+00:00
3        2017-11-12 18:00:00+00:00
4        2017-11-09 11:00:00+00:00
                    ...           
375937   2018-05-18 13:00:00+00:00
375938   2018-05-18 01:00:00+00:00
375939   2018-05-18 17:30:00+00:00
375940   2018-05-17 17:00:00+00:00
375941   2018-05-17 17:00:00+00:00
Name: time_clustering, Length: 373322, dtype: datetime64[ns, UTC]


                                                    POINT 7
For each interval, determine the number of videos, average number of likes and of dislikes.

In [8]:
interval_operations = YT_Trends.groupby("time_clustering").agg({
    "video_id": "count",  # number of videos
    "likes": "mean",      # like mean
    "dislikes": "mean"    # dislike mean
})

#rename the columns for each arithmetical operation
interval_operations = interval_operations.rename(columns={"video_id": "num_videos", "likes": "avg_likes", "dislikes": "avg_dislikes"})

print(interval_operations)

                           num_videos    avg_likes  avg_dislikes
time_clustering                                                 
2006-07-23 08:20:00+00:00           1   459.000000      152.0000
2007-03-05 16:20:00+00:00           9   336.666667        2.0000
2007-06-25 06:50:00+00:00          12   579.833333       11.5000
2007-12-03 20:50:00+00:00          16   187.937500       15.6875
2008-01-07 21:20:00+00:00          10    99.900000        2.0000
...                               ...          ...           ...
2018-06-14 02:30:00+00:00           1   853.000000       77.0000
2018-06-14 03:00:00+00:00           2  2304.500000       31.5000
2018-06-14 03:20:00+00:00           1  1414.000000       28.0000
2018-06-14 03:30:00+00:00           1  8481.000000      252.0000
2018-06-14 03:40:00+00:00           1   374.000000       24.0000

[30397 rows x 3 columns]


                                                        POINT 8
For each tag, determine the number of videos Notice that tags contains a string with several tags.

In [58]:
## WE WANT TO EXCLUDE THE CASE IN WHICH THERE IS NO TAG:
different_tags_no_nan=YT_Trends["tags"].replace("[none]", np.nan).dropna().str.split("|").explode()
videos_tags=different_tags_no_nan.value_counts()
print(videos_tags)

tags
"funny"                   14933
"comedy"                  11962
"2018"                    11029
"news"                     5955
"music"                    5590
                          ...  
"愛護"                          1
"刈る"                          1
マヤ暦 音１ 白い魔法使い フローズンマリー        1
"安住アナ"                        1
"共同通信"                        1
Name: count, Length: 886934, dtype: int64


                                                            POINT 9
Find the tags with the largest number of videos

In [59]:
print("The 50 tags with the largest number of videos:\n",videos_tags.head(50))

The 50 tags with the largest number of videos:
 tags
"funny"            14933
"comedy"           11962
"2018"             11029
"news"              5955
"music"             5590
"2017"              5500
"video"             5391
"humor"             5036
"television"        4160
"show"              4131
"review"            4012
"Pop"               3899
"vlog"              3824
"interview"         3822
"live"              3761
"food"              3629
"comedian"          3561
"funny videos"      3556
"tv"                3374
"trailer"           3245
"movie"             3230
"funny video"       3211
"how to"            3189
"Comedy"            3036
"entertainment"     3028
"rap"               2967
"celebrities"       2881
"official"          2880
"celebrity"         2872
"new"               2850
"talk show"         2811
"fun"               2744
"jokes"             2739
"hollywood"         2702
"humour"            2633
"challenge"         2621
"reaction"          2556
"film"              25

                                                    POINT 10
For each (tag, country) pair, compute average ratio likes/dislikes

In [26]:
# we only select the columns that are useful to avoid computational errors
tags = YT_Trends[["country", "tags", "likes", "dislikes"]].copy()

tags_exploded = tags.assign(tags=tags["tags"].str.split("|")).explode("tags")

tags_grouped = tags_exploded.groupby(["country", "tags"])[["likes", "dislikes"]].sum()

tags_grouped["average_ratio"] = tags_grouped["likes"] / tags_grouped["dislikes"].replace(0, np.nan)

tags_grouped = tags_grouped.dropna(subset=["average_ratio"])

print(f"Average like-dislike ratio for each tag and country pair sorted in descending order:")
tags_grouped["average_ratio"].sort_values(ascending=False)

Average like-dislike ratio for each tag and country pair sorted in descending order:


country  tags                 
RU       "Originals"              11688.0
         AT&T                     11688.0
         "DirectTV"               11688.0
         "U-Verse"                11688.0
         "AT&T AUDIENCE"          11688.0
                                   ...   
         "شبكة رؤية الإخبارية"        0.0
         "#Элджей"                    0.0
         "Эщкеренок"                  0.0
         "Эщкере"                     0.0
         "Эшкере"                     0.0
Name: average_ratio, Length: 1120369, dtype: float64

                                                        POINT 11
For each (trending_date, country) pair, the video with the largest number of views

In [27]:
idx_largest_views = YT_Trends.groupby(["trending_date", "country"])["views"].idxmax()

print(f"The video with the largest number of views for each country and trending_date paired:")
YT_Trends.loc[idx_largest_views, ["title", "views", "trending_date", "country"]].set_index(["trending_date", "country"]).sort_index()

The video with the largest number of views for each country and trending_date paired:


title  \
trending_date country                                                      
2017-11-14    CA             Ed Sheeran - Perfect (Official Music Video)   
              DE             Ed Sheeran - Perfect (Official Music Video)   
              FR             Ed Sheeran - Perfect (Official Music Video)   
              GB             Ed Sheeran - Perfect (Official Music Video)   
              IN       Tiger Zinda Hai | Official Trailer | Salman Kh...   
...                                                                  ...   
2018-06-14    JP                           Dumbo Official Teaser Trailer   
              KR                           Dumbo Official Teaser Trailer   
              MX       Courtney Hadwin: 13-Year-Old Golden Buzzer Win...   
              RU                            Семён Слепаков: Олé-Олé-Олé!   
              US       Cardi B, Bad Bunny & J Balvin - I Like It [Off...   

                          views  
trending_date country            
2017-11-14    CA       33523622  
              DE       33523622  
              FR       33523622  
              GB       33523622  
              IN       35885754  
...                         ...  
2018-06-14    JP        4427381  
              KR        4427381  
              MX        5829270  
              RU        6597033  
              US       94254507  

[1967 rows x 2 columns]

                                                        POINT 12
Divide trending_date into three columns: year, month, day

In [60]:
YT_Trends["trending_date"] = pd.to_datetime(YT_Trends["trending_date"], format='%y.%d.%m')
YT_Trends['Day'] = YT_Trends["trending_date"].dt.day
YT_Trends['Month'] = YT_Trends["trending_date"].dt.month_name()
YT_Trends['Year'] = YT_Trends["trending_date"].dt.year

print(YT_Trends.iloc[:,-3:])

        Day     Month  Year
0        14  November  2017
1        14  November  2017
2        14  November  2017
3        14  November  2017
4        14  November  2017
...     ...       ...   ...
375937   14      June  2018
375938   14      June  2018
375939   14      June  2018
375940   14      June  2018
375941   14      June  2018

[373322 rows x 3 columns]


                                                        POINT 13
For each (month, country) pair, the video with the largest number of views

In [61]:
idx_largest_views_MC=YT_Trends.groupby(["Month","country"])["views"].idxmax()

print(f"The video with the largest number of views for each month and country paired:")
YT_Trends.loc[idx_largest_views_MC,["title", "views", "Month", "Year", "country"]].set_index(["Month", "Year", "country"]).sort_index()

The video with the largest number of views for each month and country paired:


title  \
Month    Year country                                                      
April    2018 CA                           VENOM - Official Trailer (HD)   
              DE                           VENOM - Official Trailer (HD)   
              FR                    Ariana Grande - No Tears Left To Cry   
              GB       Nicky Jam x J. Balvin - X (EQUIS) | Video Ofic...   
              IN                           VENOM - Official Trailer (HD)   
...                                                                  ...   
November 2017 IN       Swag Se Swagat Song | Tiger Zinda Hai | Salman...   
              KR       Marvel Studios' Avengers: Infinity War Officia...   
              MX               Luis Fonsi, Demi Lovato - Échame La Culpa   
              RU       BTS (방탄소년단) 'MIC Drop (Steve Aoki Remix)' Offi...   
              US               Luis Fonsi, Demi Lovato - Échame La Culpa   

                           views  
Month    Year country             
April    2018 CA        53071887  
              DE        49185287  
              FR        32331284  
              GB       424538912  
              IN        53071887  
...                          ...  
November 2017 IN        40953758  
              KR        37736281  
              MX        38647842  
              RU        20565795  
              US       102012605  

[77 rows x 2 columns]

                                                        POINT 14
Read all json files with the video categories

In [72]:
folder_path = "YoutubeTrends_Data/json_data/"
countries = ['CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU', 'US']

dataframes_list = []

for country in countries:
    filename = f"{country}_category_id.json"
    full_path = folder_path + filename

    with open(full_path, 'r') as f:
        data = json.load(f)
        temp_df = pd.DataFrame(data['items'])
        temp_df['country'] = country
        dataframes_list.append(temp_df)

df_categories = pd.concat(dataframes_list, ignore_index=True)

df_categories['category_title'] = df_categories['snippet'].apply(lambda x: x['title'])

unique_categories = df_categories['category_title'].unique()

print(f"The dataframe has {len(df_categories)} rows and {len(df_categories.columns)} columns")

print("\nNumber of categories for country:")
print(df_categories['country'].value_counts())
print(unique_categories)

The dataframe has 311 rows and 6 columns

Number of categories for country:
country
US    32
CA    31
DE    31
FR    31
IN    31
GB    31
JP    31
KR    31
MX    31
RU    31
Name: count, dtype: int64
['Film & Animation' 'Autos & Vehicles' 'Music' 'Pets & Animals' 'Sports'
 'Short Movies' 'Travel & Events' 'Gaming' 'Videoblogging'
 'People & Blogs' 'Comedy' 'Entertainment' 'News & Politics'
 'Howto & Style' 'Education' 'Science & Technology' 'Movies'
 'Anime/Animation' 'Action/Adventure' 'Classics' 'Documentary' 'Drama'
 'Family' 'Foreign' 'Horror' 'Sci-Fi/Fantasy' 'Thriller' 'Shorts' 'Shows'
 'Trailers' 'Nonprofits & Activism']


                                                            POINT 15
For each country, determine how many videos have a category that is not assignable.

In [74]:
df_categories['is_assignable'] = df_categories['snippet'].apply(lambda x: x.get('assignable', True))
YT_Trends['category_id'] = YT_Trends['category_id'].astype(int)
df_categories['id'] = df_categories['id'].astype(int)

df_merged = YT_Trends.merge(
    df_categories[['id', 'country', 'is_assignable']], # we just choose the useful columns
    left_on=['category_id', 'country'],
    right_on=['id', 'country'],
    how='left'
)

not_assignable_videos = df_merged[df_merged['is_assignable'] == False]

outcome = not_assignable_videos.groupby('country').size()

print("Number of not assignable videos:")
if len(outcome) == 0:
    print("None")
else:
    print(outcome)

Number of not assignable videos:
country
CA    130
DE    110
FR    112
GB     20
IN    221
KR    167
MX      3
RU    195
US     57
dtype: int64
